<a href="https://colab.research.google.com/github/adw1029/CSCI_544/blob/main/HW1_CSCI544.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade pip setuptools wheel

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer



[nltk_data] Downloading package wordnet to /root/nltk_data...


In [10]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz

ERROR: Operation cancelled by user


## Read Data

In [11]:
import os
current_directory = os.getcwd()

In [12]:
print("Current Directory:", current_directory)


Current Directory: /content


In [13]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [14]:
google_drive_directory = '/content/drive/My Drive/CSCI_544/hw1'
os.chdir(google_drive_directory)

In [15]:
current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: /content/drive/My Drive/CSCI_544/hw1


In [16]:
gz_file_path = 'amz_data.gz'

## Keep Reviews and Ratings

In [17]:
file_path = 'amz_data.tsv'
data = pd.read_csv(file_path, sep='\t', usecols=['review_body', 'star_rating'])

<ipython-input-17-04b759942888>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t', usecols=['review_body', 'star_rating'])


In [18]:
print(data)

        star_rating                                        review_body
0                 5                                     Great product.
1                 5  What's to say about this commodity item except...
2                 5    Haven't used yet, but I am sure I will like it.
3                 1  Although this was labeled as &#34;new&#34; the...
4                 4                    Gorgeous colors and easy to use
...             ...                                                ...
2640347           4  I can't live anymore whithout my Palm III. But...
2640348           4  Although the Palm Pilot is thin and compact it...
2640349           4  This book had a lot of great content without b...
2640350           5  I am teaching a course in Excel and am using t...
2640351           5  A very comprehensive layout of exactly how Vis...

[2640352 rows x 2 columns]


Separate out review and rating, dropped null values

In [19]:
df_RevRat = data[data['star_rating'] != 3]

In [20]:
contains_null = df_RevRat[['star_rating', 'review_body']].isnull().any()
contains_null

star_rating    True
review_body    True
dtype: bool

In [21]:
df_RevRat = data.dropna(subset=['star_rating', 'review_body'])

In [22]:
df_RevRat

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640347,4,I can't live anymore whithout my Palm III. But...
2640348,4,Although the Palm Pilot is thin and compact it...
2640349,4,This book had a lot of great content without b...
2640350,5,I am teaching a course in Excel and am using t...


In [23]:
df_RevRat['star_rating'] = pd.to_numeric(df_RevRat['star_rating'], errors='coerce')


<ipython-input-23-835b95727621>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RevRat['star_rating'] = pd.to_numeric(df_RevRat['star_rating'], errors='coerce')


In [24]:
df_RevRat['star_rating'].dtypes

dtype('float64')

In [25]:
df_RevRat['Sentiment'] = df_RevRat['star_rating'].apply(lambda x: 1 if x > 3 else 0)

<ipython-input-25-940d36b9d541>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_RevRat['Sentiment'] = df_RevRat['star_rating'].apply(lambda x: 1 if x > 3 else 0)


In [26]:
df_RevRat['Sentiment']

0          1
1          1
2          1
3          0
4          1
          ..
2640347    1
2640348    1
2640349    1
2640350    1
2640351    1
Name: Sentiment, Length: 2640255, dtype: int64

check if there's null values

In [27]:
contains_null = df_RevRat[['star_rating', 'review_body']].isnull().any()
contains_null

star_rating    False
review_body    False
dtype: bool

## 20000 samples from positive (100000) and negative (100000) reviews

In [28]:
pos_rev = df_RevRat[df_RevRat['Sentiment'] == 1].sample(100000, random_state=42)
neg_rev = df_RevRat[df_RevRat['Sentiment'] == 0].sample(100000, random_state=42)
df_small = pd.concat([pos_rev, neg_rev])

In [29]:
df_small.sample(3)

,star_rating,review_body,Sentiment
2475813,1.0,I purchased this ink to use in the Brother DCP...,0
1859239,3.0,Virtually all box cutters I have previously pu...,0
2311590,5.0,So far so good. I purchsed it 20 days ago. It ...,1


# Data Cleaning



# avg review length before cleaning

In [30]:
avglen_before = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_before)


Average text length: 315.554375


reviews to lower case

In [31]:
df_small['review_body'] = df_small['review_body'].str.lower()


remove URL and HTML

In [111]:
import html

In [112]:
url_pattern = re.compile(r'https?://\S+|www\.\S+')

In [113]:
def remove_urls(text):
    return url_pattern.sub(r'', text)

# Apply the functions to the review column
df_small['review_body'] = df_small['review_body'].apply(remove_urls)

In [114]:
df_small['review_body'] = df_small['review_body'].apply(html.unescape)

remove non-alpha

In [115]:
df_small['review_body'] = df_small['review_body'].str.replace('[^a-zA-Z]', ' ', regex=True)

remove extra spaces

In [116]:
df_small['review_body'] = df_small['review_body'].str.replace(r'\s+', ' ', regex=True)

contractions

In [117]:
pip install contractions

In [118]:
import contractions


In [119]:
df_small['review_body'] = df_small['review_body'].apply(contractions.fix)

###average review length after cleaning

In [120]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 184.67001


# Pre-processing

In [121]:
from nltk.corpus import stopwords

### 3 sample reviews before pre-processing / Avg length

In [122]:
print(df_small['review_body'].head(3))

386895     reviewed packaging previously gave star replac...
2514581    got tired searching place envelope decided buy...
785896                   best faber castell never dissapoint
Name: review_body, dtype: object


In [123]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 184.67001


### remove the stop words

In [124]:
nltk.download('stopwords')

stp_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [125]:
df_small['review_body'] = df_small['review_body'].apply(lambda x: ' '.join([word for word in str(x).split() if word.lower() not in stp_words]))


## perform lemmatization  

In [126]:
from nltk.stem import WordNetLemmatizer

In [127]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [128]:
lemmatizer = WordNetLemmatizer()
df_small['review_body'] = df_small['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))


In [129]:
print(df_small['review_body'].head(3))

386895     reviewed packaging previously gave star replac...
2514581    got tired searching place envelope decided buy...
785896                   best faber castell never dissapoint
Name: review_body, dtype: object


In [130]:
avglen_after = df_small['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length:", avglen_after)

Average text length: 184.520675


# TF-IDF Feature Extraction

#split testing (20%) and training (80%) set


In [131]:
vectorizer = TfidfVectorizer(max_features = 10000)
tfidf_mat = vectorizer.fit_transform(df_small['review_body'])

In [132]:
tfidf_mat

<200000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 4505096 stored elements in Compressed Sparse Row format>

# Perceptron

In [133]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [134]:
X = tfidf_mat
y = df_small['Sentiment']

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [136]:
perceptron = Perceptron(random_state= 42, eta0=1.0).fit(X_train, y_train)

In [137]:
prec_train = perceptron.predict(X_train)
prec_test = perceptron.predict(X_test)

In [138]:
# Training Metrics
perc_train_acc = accuracy_score(y_train, prec_train)
perc_train_f1 = f1_score(y_train, prec_train, average = 'weighted')
perc_train_pre = precision_score(y_train, prec_train, average = 'weighted')
perc_train_recall = recall_score(y_train, prec_train, average = 'weighted')

# Testing Metrics
perc_test_acc = accuracy_score(y_test, prec_test)
perc_test_f1 = f1_score(y_test, prec_test, average = 'weighted')
perc_test_precision = precision_score(y_test, prec_test, average = 'weighted')
perc_test_recall = recall_score(y_test, prec_test, average = 'weighted')

In [139]:
print('Training Accuracy Score :', perc_train_acc)
print('Training F1 Score :', perc_train_f1)
print('Training Precision Score :',perc_train_pre)
print('Training Recall Score :', perc_train_recall)

print('\nTesting Accuracy Score :', perc_test_acc)
print('Testing F1 Score :', perc_test_f1)
print('Testing Precision Score :', perc_test_precision)
print('Testing Recall Score :', perc_test_recall)

Training Accuracy Score : 0.8212375
Training F1 Score : 0.8212360419560973
Training Precision Score : 0.8212475764994284
Training Recall Score : 0.8212375

Testing Accuracy Score : 0.799675
Testing F1 Score : 0.7996750016276408
Testing Precision Score : 0.7996750042548754
Testing Recall Score : 0.799675


# SVM

In [140]:
from sklearn.datasets import make_classification
from sklearn import svm
from sklearn.linear_model import SGDClassifier

In [141]:
svm_sgd = SGDClassifier(loss='hinge', alpha = 0.001, max_iter = 1000, random_state=42)

In [142]:
svm_sgd.fit(X_train, y_train)

SGDClassifier(alpha=0.001, random_state=42)

In [143]:
SVM_train = svm_sgd.predict(X_train)
SVM_test = svm_sgd.predict(X_test)

In [144]:
# Training Metrics
svm_train_acc = accuracy_score(y_train, SVM_train)
svm_train_f1 = f1_score(y_train, SVM_train, average = 'weighted')
svm_train_pre = precision_score(y_train, SVM_train, average = 'weighted')
svm_train_recall = recall_score(y_train, SVM_train, average = 'weighted')

# Testing Metrics
svm_test_acc = accuracy_score(y_test, SVM_test)
svm_test_f1 = f1_score(y_test, SVM_test, average = 'weighted')
svm_test_precision = precision_score(y_test, SVM_test, average = 'weighted')
svm_test_recall = recall_score(y_test, SVM_test, average = 'weighted')

In [145]:
print('Training Accuracy Score :', svm_train_acc)
print('Training F1 Score :', svm_train_f1)
print('Training Precision Score :',svm_train_pre)
print('Training Recall Score :', svm_train_recall)

print('\nTesting Accuracy Score :', svm_test_acc)
print('Testing F1 Score :', svm_test_f1)
print('Testing Precision Score :', svm_test_precision)
print('Testing Recall Score :', svm_test_recall)

Training Accuracy Score : 0.81751875
Training F1 Score : 0.8165889649022962
Training Precision Score : 0.8241009820413489
Training Recall Score : 0.81751875

Testing Accuracy Score : 0.815575
Testing F1 Score : 0.8144864646314589
Testing Precision Score : 0.8231149147144271
Testing Recall Score : 0.815575


# Logistic Regression

In [146]:
from sklearn.linear_model import LogisticRegression


In [147]:
logReg = LogisticRegression(random_state=42, max_iter=1000)

In [148]:
logReg.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [149]:
log_train = logReg.predict(X_train)
log_test = logReg.predict(X_test)

In [150]:
# Training Metrics
log_train_acc = accuracy_score(y_train, log_train)
log_train_f1 = f1_score(y_train, log_train, average = 'weighted')
log_train_pre = precision_score(y_train, log_train, average = 'weighted')
log_train_recall = recall_score(y_train, log_train, average = 'weighted')

# Testing Metrics
log_test_acc = accuracy_score(y_test, log_test)
log_test_f1 = f1_score(y_test, log_test, average = 'weighted')
log_test_precision = precision_score(y_test, log_test, average = 'weighted')
log_test_recall = recall_score(y_test, log_test, average = 'weighted')

In [151]:
print('Training Accuracy Score :', log_train_acc)
print('Training F1 Score :', log_train_f1)
print('Training Precision Score :',log_train_pre)
print('Training Recall Score :', log_train_recall)

print('\nTesting Accuracy Score :', log_test_acc)
print('Testing F1 Score :', log_test_f1)
print('Testing Precision Score :', log_test_precision)
print('Testing Recall Score :', log_test_recall)

Training Accuracy Score : 0.86563125
Training F1 Score : 0.8656282184951241
Training Precision Score : 0.8656649746480635
Training Recall Score : 0.86563125

Testing Accuracy Score : 0.851675
Testing F1 Score : 0.8516616169584751
Testing Precision Score : 0.8517963961102458
Testing Recall Score : 0.851675


# Naive Bayes

In [152]:
from sklearn.naive_bayes import MultinomialNB

In [153]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [154]:
nb_train = nb.predict(X_train)
nb_test = nb.predict(X_test)

In [155]:
# Training Metrics
nb_train_acc = accuracy_score(y_train, nb_train)
nb_train_f1 = f1_score(y_train, nb_train, average = 'weighted')
nb_train_pre = precision_score(y_train, nb_train, average = 'weighted')
nb_train_recall = recall_score(y_train, nb_train, average = 'weighted')

# Testing Metrics
nb_test_acc = accuracy_score(y_test, nb_test)
nb_test_f1 = f1_score(y_test, nb_test, average = 'weighted')
nb_test_precision = precision_score(y_test, nb_test, average = 'weighted')
nb_test_recall = recall_score(y_test, nb_test, average = 'weighted')

In [156]:
print('Training Accuracy Score :', nb_train_acc)
print('Training F1 Score :', nb_train_f1)
print('Training Precision Score :',nb_train_pre)
print('Training Recall Score :', nb_train_recall)

print('\nTesting Accuracy Score :', nb_test_acc)
print('Testing F1 Score :', nb_test_f1)
print('Testing Precision Score :', nb_test_precision)
print('Testing Recall Score :', nb_test_recall)

Training Accuracy Score : 0.8319875
Training F1 Score : 0.831977811309056
Training Precision Score : 0.8320629918963321
Training Recall Score : 0.8319875

Testing Accuracy Score : 0.822425
Testing F1 Score : 0.8224241845946164
Testing Precision Score : 0.8224322607721267
Testing Recall Score : 0.822425
